In [8]:
class Counter:
    """I count. That is all."""
    
    some_attribute = 42
    _internal_attribute = []
    __very_internal_attribute = []


    def __init__(self, initial=0):
        self.value = initial
    
    
    def increment(self):
        self.value += 1
    

    def get(self):
        return self.value

c = Counter(42)
c.increment()
c.get()
c.some_attribute
c._internal_attribute
Counter._Counter__very_internal_attribute


[]

In [19]:
from collections import deque

class MemorizingDict(dict):
    history = deque(maxlen=10)


    def set (self, key, value):
        self.history.append(key)
        self[key] = value

    
    def get_history(self):
        return self.history


d = MemorizingDict({"foo": 42})
d.set("baz", 100500)
print(d.get_history()[0])

d = MemorizingDict()
d.set("boo", 500100)
d.get_history()

baz


deque(['baz', 'boo'])

In [1]:
Counter.__dict__
Counter.__class__

NameError: name 'Counter' is not defined

In [3]:
class Noop:
    __slots__ = ["some_attribute"]  #лучше NamedTuple

noop = Noop()
noop.some_attribute = 42
#noop.some_other_attribute = 100500

In [7]:
import os


class Path:
    def __init__(self, current):
        self.current = current
    
    
    def __repr__(self):
        return "Path({})".format(self.current)

    
    @property
    def parent(self):
        return Path(os.path.dirname(self.current))


p = Path("./tutorial/examples/some_file.txt")
p.parent

Path(./tutorial/examples)

In [13]:
class BigDataModel:
    def __init__(self):
        self._params = []
    

    @property
    def params(self):
        return self._params
    

    @params.setter
    def params(self, new_params):
        assert all(map(lambda p: p > 0, new_params))
        self._params = new_params

    
    @params.deleter
    def params(self):
        del self._params


model = BigDataModel()
model.params = [0.1, 0.5, 0.4]
model.params

[0.1, 0.5, 0.4]

In [18]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial


class OtherCounter(Counter):
    def get(self):
        return self.value


c = OtherCounter()
c.get()
c.value

0

In [20]:
class Temp:
    all_counters = []


    def __init__(self, initial=0):
        self.__class__.all_counters.append(self)
        self.value = initial


class OtherTemp(Temp):
    def __init__(self, initial=0):
        self.initial = initial
        super().__init__(initial)

ot = OtherTemp()
vars(ot)

{'initial': 0, 'value': 0}

In [25]:
isinstance(OtherCounter(), Counter)  #проверка экземпляра
issubclass(OtherCounter, Counter)    #проверка наследника


True

In [26]:
class ThreadSafeMixin:
    get_lock = ...


    def increment(self):
        with self.get_lock():
            super().increment()

    
    def get(self):
        with self.get_lock():
            return super().get()


class ThreadSafeCounter(ThreadSafeMixin, Counter):
    pass

In [27]:
def thread_safe(cls):
    orig_increment = cls.increment
    orig_get = cls.get

    def increment(self):
        with self.get_lock():
            orig_increment(self)

    
    def get(self):
        with self.get_lock():
            return orig_get(self)

    
    cls.get_lock = ...
    cls.increment = increment
    cls.get = get
    return cls

In [35]:
import functools


def singleton(cls):
    instance = None

    @functools.wraps(cls)
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance

    return inner

@singleton
class Noop:
    "I do noting at all."


print(id(Noop()), id(Noop()))


140356178382224 140356178382224


In [39]:
import warnings


def deprecated(cls):
    orig_init = cls.__init__

    @functools.wraps(cls.__init__)
    def new_init(self, *args, **kwargs):
        warnings.warn(
            cls.__name__ + " is deprecated.",
            category=DeprecationWarning)
        orig_init(self, *args, **kwargs)
    
    cls.__init__ = new_init
    return cls


@deprecated
class Counter:
    def __init__(self, initial=0):
        self.value = initial


c = Counter()

/tmp/ipykernel_420/1536756259.py:9: DeprecationWarning: Counter is deprecated.
  warnings.warn(


In [41]:
class Noop:
    pass

Noop().foobar

AttributeError: 'Noop' object has no attribute 'foobar'

In [42]:
class Noop:
    def __getattr__(self, name):
        return name

Noop().foobar

'foobar'